In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Get the data**

In [3]:
## Titanic competition in Kaggle
## data downloaded from Kaggle.
from ast import Not
from statistics import mean
from matplotlib import pyplot as plt
import pandas
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns


train = pandas.read_csv('/kaggle/input/titanic/train.csv')
train.columns  

# 1. EDA

**1.1** Check the data, including graphs and correlations, both numerical and categorical.

In [4]:
train.head()

The *Name* variable is not informative. Change to surname (to indicate family tie) 
and title (to indicate marriage and age) 

In [5]:
surname = []
title = []
for txt in train['Name']:
      txt1 = txt.split(',')[0]
      txt2 = (txt.split(",")[1]).split(".")[0]
      surname.append(txt1)
      title.append(txt2)

train['Surname'] = pandas.DataFrame(surname)
train['Title'] = pandas.DataFrame(title, dtype='category')



**An illustraion of data**

In [6]:
import seaborn as sns
sns.pairplot(train, hue='Survived')

From the graph above, among all numeric variables, Pclass 1, minor age, 
2 SibSp, and extreme high fare seem to have high survival probability.


In [7]:
corr_matrix = train.corr()
print(corr_matrix['Survived'].sort_values(ascending=False))


It looks that all numeric variables except PassengerId are correlated to Survived.
Check some categorical variables too.

In [7]:
sns.histplot(data=train, x='Sex',hue='Survived')

In [8]:
sns.histplot(data=train, x='Title',hue='Survived')

**Too many kind of Titles**. Decrease.

In [8]:
train.loc[:,'Title'].value_counts()

title_list = ((train.loc[:,'Title'].value_counts())[:6]).index.to_list()

for ind in train.index:
    if not(train.loc[ind, 'Title'] in title_list):
        train.loc[ind, 'Title'] = "Others"

train.loc[:,'Title'].value_counts()

sns.histplot(data=train, x='Title',hue='Survived')

***Title* looks much better now.**

**1.2 Missing values**

In [9]:
# missing values in train
np.sum(train.isna(), axis=0)

We split the train data to own train and test set 
before treatment of missing values, in order not to overfit.
Variables *Name* and *PassengerID* is droped.

In [10]:
X = train.drop(['Survived','Name', 'PassengerId'],axis=1)
y = (train.copy())['Survived']

# Split the (training) data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4686)

# missing values in X_train
np.sum(X_train.isna(), axis=0)

Fix missing values with means of corresponding group. Use the powerful *groupby* in *pandas*.
**age => Title and Sex**

**Embarked => Pclass and Sex**

In [11]:
X_train["Age"] = X_train["Age"].fillna(X_train.groupby(['Sex','Title'])['Age'].transform('mean'))
X_test["Age"] = X_test["Age"].fillna(X_test.groupby(['Sex','Title'])['Age'].transform('mean'))

# There is no suitable function of "Most-frequent" for 'transform'.

fill_values = ((X_train.groupby(['Pclass','Sex'])['Embarked']).describe())['top']

for ind in (X_train.loc[X_train['Embarked'].isna()]).index:
    X_train.loc[ind, 'Embarked'] = fill_values[(X_train.loc[ind,'Pclass'], X_train.loc[ind,'Sex'])]


There are too many missing values in "Cabin" with uncertain impact 
to the accuracy after imputation. (It seems improve a little bit 
after evaluation.)

# 2. Preprocessing
Simple imputation for Cabin, Encode and Pipeline
Import packages from SciKit-Learn

In [12]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

**Set up preprocessing with Pipeline (ColumnTransformer)**

In [13]:
Impute1 = SimpleImputer(strategy='constant')
Impute2 = SimpleImputer(strategy="most_frequent")
Encode = OneHotEncoder(handle_unknown="ignore")
Scale = StandardScaler()

Num_attr1 = ['Fare']
Num_attr2 = ['Age']
Attrib2 = ['Pclass', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Surname', 'Title']

#Attrib1 = X_train.select_dtypes(include = np.number).columns.to_list()
#Attrib2 = X_train.select_dtypes(exclude = np.number).columns.to_list()

num_pipiline1 = Pipeline(steps=[
      ('poly', PolynomialFeatures()),
    ('scale1', StandardScaler())
])

num_pipiline2 = Pipeline(steps=[
    ('scale1', StandardScaler())
])

cat_pipeline1 = Pipeline(steps=[
    ('enco', Encode)
])

total_pipeline = ColumnTransformer(transformers=[
    ('num1', num_pipiline1, Num_attr1),
    ('num2', num_pipiline2, Num_attr2),
    ('cat', cat_pipeline1, Attrib2)
], remainder="passthrough")


# 3. Logistic regression
Logistic regression is the most natural choice for this competition.

In [14]:
Logit = Pipeline(steps=[ 
    ('preprocess', total_pipeline),
    ('logit', LogisticRegression(penalty='l2', C=1, max_iter=1000, solver="liblinear") )
]
)

titanic_logit = Logit.fit(X_train, y_train)
titanic_pred = Logit.predict(X_train)
print(classification_report(y_train, titanic_pred))
# accuracy 0.94

It is not bad. However, only for the train data.

In [15]:
cross_logit = cross_val_score(Logit, X_train, y_train, cv=10)
print("Cross_val_score is ", np.mean(cross_logit))
# ~ 0.8428312980551788
print("For own created test dataset")
print(classification_report(y_test, Logit.predict(X_test)))
# 0.85

It is a rather good accuracy (look at the leaderboard). We can use GridSearch to tune parameters. It is not run to save running time. One option is as follows.

dist = {
      'preprocess__num1__poly__degree': range(1,4),
    'logit__C': range(0,1000,50), 
    'logit__penalty': ['l2','l1']
}

lr_clf = RandomizedSearchCV(Logit, param_distributions=dist, n_iter=10, cv=3)

lr_clf.fit(X_train, y_train)

print(classification_report(y_train, lr_clf.predict(X_train)))
#1 

'''
best_params_:
'preprocess__num1__poly__degree': 3, 'logit__penalty': 'l2', 'logit__C': 900
'''

print(np.mean(cross_val_score(lr_clf, X_train, y_train, cv=10)))
#0.842740841248304  Overfitting.

print(classification_report(y_test, lr_clf.predict(X_test)))
#~0.83


It is not better than the default model. We can try *SDGClassifier*.

In [16]:
Logit2 = Pipeline(steps=[ 
    ('preprocess', total_pipeline),
    ('logit2', SGDClassifier(penalty='l2', alpha=0.0001, max_iter=1000, loss="log") )
]
)
titanic_logit = Logit2.fit(X_train, y_train)
print("Accuracy train: ", classification_report(y_train, Logit2.predict(X_train)))
# 1
print("Cross_val_score is ", np.mean(cross_val_score(Logit2, X_train, y_train, cv=10)))
# ~0.83
print("Accuracy for own test data:")
print(classification_report(y_test, Logit2.predict(X_test)))
# 0.78

Not better either. 

**Check Feature permutation importance to select feature**

In [18]:
fpi = permutation_importance(estimator=Logit, X= X_train, y= y_train, n_repeats=8, random_state=4686)

sns.barplot(
    data=pandas.DataFrame({
        'permutation importance': fpi['importances_mean'],
        'feature': X_train.columns
    }).sort_values('permutation importance', ascending=False),
    y='feature', x='permutation importance'
)
plt.title('Logistic: feature permutation importance on train data')


The graph shows that Embarked, Fare (surprisingly), Parch, SibSp, and Cabin
have low permutation importance. Try to leave them.

In [19]:
Num_attr2 = ['Age']
Cat_logit = ['Pclass', 'Sex', 'Ticket', 'Surname', 'Title']

total_lg = ColumnTransformer(transformers=[
    ('num2', num_pipiline2, Num_attr2),
    ('cat', cat_pipeline1, Cat_logit)
], remainder="drop")

Logit3 = Pipeline(steps=[ 
    ('preprocess', total_lg),
    ('logit', LogisticRegression(penalty='l2', C=1000, max_iter=1000, solver="liblinear") )
]
)

Logit3.fit(X_train, y_train)
print("Accuracy for the train data:")
print(classification_report(y_train, Logit3.predict(X_train)))
# 1

print("Cross_val_score: ", np.mean(cross_val_score(Logit3, X_train, y_train, cv=10)))
# ~0.84
print("Accuracy for the test data set:\n")
print(classification_report(y_test, Logit3.predict(X_test)))
# 0.80



And run GridSearch again. 

**It is no problem with error messages (*some_fits_failed_message*) below during some of Grid Searches.**

In [20]:
dist3 = {
    'logit__C': range(0,1000,50), 
    'logit__penalty': ['l2','l1']
}

Logit3_chosen = RandomizedSearchCV(Logit3, param_distributions=dist3, n_iter=10, cv=5)
Logit3_chosen.fit(X_train, y_train)
print("Accuracy for the train data:")
print(classification_report(y_train, Logit3_chosen.predict(X_train)))
#1
print("Cross_val_score is: ", np.mean(cross_val_score(Logit3_chosen, X_train, y_train, cv=10)))
# ~0.84
print("Accuracy for the test data:")
print(classification_report(y_test, Logit3_chosen.predict(X_test)))
# 0.85

# 3. Support Vector Machine

In [21]:
from sklearn.svm import SVC

svc = Pipeline(steps=[ 
    ('preprocess', total_pipeline),
    ('svm', SVC(C=100, kernel="linear"))
])

titan_svc = svc.fit(X_train, y_train)
print("SVM Accuracy for the train data:")
print(classification_report(y_train, svc.predict(X_train)))
##  1.  
print("Cross_val_score: ", np.mean(cross_val_score(svc, X_train, y_train, cv=5)))
# ~0.84
print("SVM Accuracy for own test data:")
print(classification_report(y_test, svc.predict(X_test)))
## 0.83


To run GridSearch to tune paremeters:

dist_svc= { 
    "svm__C": range(1, 1000, 100),
    "svm__kernel": ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
}

svc_chosen = RandomizedSearchCV(svc, param_distributions=dist_svc, n_iter=50, random_state=4686)
svc_chosen.fit(X_train, y_train)

#svc_chosen.best_params_
#{'svm__kernel': 'linear', 'svm__C': 1}
#It takes about one hour to run. n*m*log_2(m)

We use direct the best parameters.

In [22]:
svc_chosen = Pipeline(steps=[ 
    ('preprocess', total_pipeline),
    ('svm', SVC(C=1, kernel="linear"))
])

svc_chosen.fit(X_train, y_train)
pred_svc_rs = svc_chosen.predict(X_train)
print("SVM Accuracy for the train data:")
print(classification_report(y_train, pred_svc_rs))
# 1 
print("Cross_val_score: ", np.mean(cross_val_score(svc_chosen, X_train, y_train, cv=10)))
# 0.8487788331071913
print("SVM Accuracy for test data:")
print(classification_report(y_test, svc_chosen.predict(X_test)))
#0.84

**3.1 Permutation Importance**

See https://scikit-learn.org/stable/modules/permutation_importance.html for referee on feature permutation importance.

In [23]:
fpi_svm = permutation_importance(estimator=svc_chosen, X= X_train, y= y_train, n_repeats=8, random_state=4686)

sns.barplot(
    data=pandas.DataFrame({
        'permutation importance': fpi_svm['importances_mean'],
        'feature': X_train.columns
    }).sort_values('permutation importance', ascending=False),
    y='feature', x='permutation importance'
)
plt.title('SVM Feature permutation importance on X_train')

*Embarked* and *Parch* have low importance. Surprisingly, *Pclass, Fare, Age* are not high either. It comfuses somehow. We leave *Embarked* and *Parch*, but not others.

In [25]:
num1_svm = Num_attr1
num2_svm = Num_attr2
cat1 = ['Sex', 'Ticket', 'Surname', 'Title', 'Cabin', 'Pclass']

total_svm = ColumnTransformer(transformers=[ 
    ('num1', num_pipiline1, num1_svm),
    ('num2', num_pipiline2, num2_svm),
    ('cat1', cat_pipeline1, cat1)
], remainder="drop")

svc2 = Pipeline(steps=[ 
    ('prep', total_svm),
    ('svc2',SVC(C=1, kernel="linear"))
])

svc2.fit(X_train, y_train)
print("SVC 2 accuracy for train data: ")
print(classification_report(y_train, svc2.predict(X_train)))
#1
print(np.mean(cross_val_score(svc2, X_train, y_train, cv=10)))
# 0.85
print("SVC 2 accuracy for test data: ")
print(classification_report(y_test, svc2.predict(X_test)))
# 0.86


**A litte improvement after feature selection.**

# 4. Random Forest

We repeat the process as for Random Forest. Cross Validation, GridSearch, feature selection, and so on.

In [27]:
random_forest = Pipeline(steps=[ 
    ('preprocess', total_pipeline),
    ('rf', RandomForestClassifier(criterion="entropy", min_samples_leaf=1, random_state=4686))
])

titan_rf = random_forest.fit(X_train, y_train)
print("Random Forest Accuracy for train data:")
print(classification_report(y_train, random_forest.predict(X_train)))
#  1
print("Cross_val_score: ", np.mean(cross_val_score(random_forest, X_train, y_train, cv=10)))
# ~0.83 

print("Random Forest Accuracy for own test data:")
print(classification_report(y_test, random_forest.predict(X_test)))
# 0.86

Grid Search: 

dist_rf = {
      'preprocess__num1__poly__degree': range(2,4),
      'rf__criterion': ['gini','entropy'],
      'rf__max_depth': np.arange(5, 10),
      'rf__min_samples_leaf': np.arange(1, 15, 2)
}

rf_chosen = GridSearchCV(random_forest, param_grid=dist_rf, cv=10)
rf_chosen.fit(X_train, y_train)
print(classification_report(y_train, rf_chosen.predict(X_train)))
#0.86

'''
{'preprocess__num1__poly__degree': 2,
 'rf__criterion': 'gini',
 'rf__max_depth': 9,
 'rf__min_samples_leaf': 1}
'''

print(classification_report(y_test, rf_chosen.predict(X_test)))
#0.80 

Not as good as the default model above; thus not used in the end. 


**Mean Decrease Impurity**

Another way to check feature importance for Decision trees.

In [29]:
feature_rf_num1 = random_forest['preprocess'].named_transformers_['num1'].named_steps['poly'].get_feature_names_out()
feature_rf_num2 = random_forest['preprocess'].named_transformers_['num2'].get_feature_names_out()
feature_rf_cat1 = random_forest['preprocess'].named_transformers_['cat'].get_feature_names_out()
feature_rf_re = X_train.columns[random_forest['preprocess'].transformers_[-1][-1]]

feature_rf = np.r_[feature_rf_num1, feature_rf_num2, feature_rf_cat1, feature_rf_re]

mdi = random_forest.named_steps['rf'].feature_importances_

sns.barplot(
    data=pandas.DataFrame({
        'mean decrease impurity': mdi,
        'feature': feature_rf
    }).sort_values('mean decrease impurity', ascending=False).iloc[0:11],
    y='feature', x='mean decrease impurity'
)
plt.title('Random Forest: mean decrease impurity on X_train')

MDI for all variables are not so high. Check Permutation importances too.

In [30]:
fpi_rf = permutation_importance(estimator=random_forest, X= X_train, y= y_train, n_repeats=8, random_state=4686)

fpi_rf_df = pandas.DataFrame({
        'permutation importance': fpi_rf['importances_mean'],
        'feature': X_train.columns
    }).sort_values('permutation importance', ascending=False)

print(fpi_rf_df)

sns.barplot(data=fpi_rf_df, y='feature', x='permutation importance')

plt.title('permutation importance Random Forest on X_train')


Permutation importances are low for some variables. We test to leave alone *SibSp* and *Parch*.

In [33]:
rf_chosen2 = Pipeline(steps=[ 
    ('prep2', total_svm), #same variables as SVM
    ('rf2', RandomForestClassifier(criterion="entropy",max_depth=9,  min_samples_leaf=1, random_state=4686)) 
    #max_depth from the Grid Search above.
])

rf_chosen2.fit(X_train, y_train)
print("New Random Forest Accuracy for train data:")
print(classification_report(y_train, rf_chosen2.predict(X_train)))
# 0.86
print('Cross_val_Score is: ', np.mean(cross_val_score(rf_chosen2, X_train, y_train, cv=8)))
# 0.7994298623063684
print("New Random Forest Accuracy for own test data:")
print(classification_report(y_test, rf_chosen2.predict(X_test)))
# 0.77 Not better than before.


# 5. K-Neighbors classifier

In [34]:
from sklearn.neighbors import KNeighborsClassifier

knc = Pipeline(steps=[ 
    ('preprocess', total_pipeline),
    ('knclf', KNeighborsClassifier())
])

knc.fit(X_train, y_train)
print("K-neighbors classifier Accuracy for train data: ")
print(classification_report(y_train, knc.predict(X_train)))
#0.87
print()
print('Cross_val_score is: ', np.mean(cross_val_score(knc, X_train, y_train)))
# ~0.81
print("K-neighbors classifier Accuracy for own test data: ")
print(classification_report(y_test, knc.predict(X_test)))
# 0.80


**Tune parameters.**


In [38]:
disc_knc = {
     'preprocess__num1__poly__degree': range(2,4),
    "knclf__n_neighbors": range(0,100,5),
    "knclf__weights":['uniform','distance']
}

knc_grid = RandomizedSearchCV(knc, param_distributions=disc_knc, cv=10, random_state=4686)
knc_grid.fit(X_train, y_train)

print('K-neighbors classifier Accuracy for train data:')
print(classification_report(y_train, knc_grid.predict(X_train)))
# 1
print()
print('Cross_val_score: ', np.mean(cross_val_score(knc_grid, X_train, y_train, cv=10), 0))
# ~ 0.8. 
print('K-neighbors classifier Accuracy for own test data:')
print(classification_report(y_test, knc_grid.predict(X_test)))
#0.81


In [39]:
# Feature permutation importances

fpi_knc = permutation_importance(estimator=knc_grid, X= X_train, y= y_train, n_repeats=8, random_state=4686)

fpi_knc_df = pandas.DataFrame({
        'permutation importance': fpi_knc['importances_mean'],
        'feature': X_train.columns
    }).sort_values('permutation importance', ascending=False)


sns.barplot(data=fpi_knc_df, y='feature', x='permutation importance')
plt.title('permutation importance K-Neighbors Classifier on X_train')
print(fpi_knc_df)


There are no significant differences of permutation importances.

# 6. Ensemble algorithm

Try ensemble algorithms such as Voting Classifier.

In [40]:
from sklearn.ensemble import VotingClassifier

# Equal weights
vote_clf = VotingClassifier(
    estimators= [('lr', Logit3_chosen), ('knc', knc_grid), ('svc', svc2), ('rf', random_forest)],
    voting='hard'
)

vote_clf.fit(X_train, y_train)
print('Voting classifier Accuracy for train data: ')
print(classification_report(y_train, vote_clf.predict(X_train)))
# 1
print()
print('Cross_val_score: ', np.mean(cross_val_score(vote_clf, X_train, y_train, cv=10), 0))
# 0.8427634554500226
print()
print('Voting classifier Accuracy for own test data: ')
print(classification_report(y_test, vote_clf.predict(X_test)))
#0.86


*Soft*-voting is unavailable since some of the algorithms don't have *predict_proba*. 
We try to skip the K-neighbors classifier since it is the worst one among others.

In [41]:
vote_clf2 = VotingClassifier(
    estimators= [('lr', Logit3_chosen), ('svc', svc2), ('rf', random_forest)],
    voting='hard'
)

vote_clf2.fit(X_train, y_train)
print("Voting classifier 2 Accuracy for train data: ")
print(classification_report(y_train, vote_clf2.predict(X_train)))
# 1
print()
print('Cross_val_score: ', np.mean(cross_val_score(vote_clf2, X_train, y_train, cv=10), 0))
# ~0.84
print()
print("Voting classifier 2 Accuracy for test data: ")
print(classification_report(y_test, vote_clf2.predict(X_test)))
#0.86


# 7. Submit the result

In [42]:
test = pandas.read_csv('/kaggle/input/titanic/test.csv')
# pred_test = random_forest.predict(test)
np.sum(test.isna(),0)
# Error: "Fare" has a missing value, in contrast to X_train and X_test.

surname = []
title = []
for txt in test['Name']:
      txt1 = txt.split(',')[0]
      txt2 = (txt.split(",")[1]).split(".")[0]
      surname.append(txt1)
      title.append(txt2)

test['Surname'] = pandas.DataFrame(surname)
test['Title'] = pandas.DataFrame(title, dtype='category')

## Adjust Title as for Train
title_test = ((test.loc[:,'Title'].value_counts())[:6]).index.to_list()

for ind in test.index:
    if not(test.loc[ind, 'Title'] in title_test):
        test.loc[ind, 'Title'] = "Others"

test.loc[:,'Title'].value_counts()

test["Age"] = test["Age"].fillna(test.groupby(['Sex','Title'])['Age'].transform('mean'))
# Not all missing values can be filled by group-by means; some might be missing.
# test['Age'] = test['Age'].fillna(test['Age'].mean())

test['Fare'] = test['Fare'].fillna(test.groupby(['Pclass'], dropna=True)['Fare'].transform('mean'))

test2 = test.drop(['Name', 'PassengerId'], axis=1)
pred_test = vote_clf2.predict(test2)

res1 = (test.copy())['PassengerId']
result = (pandas.DataFrame(res1)).assign(Survived = pred_test)
result.to_csv('submission.csv',index=False)

## The end.